In [18]:
import pandas as pd
import numpy as np
from pathlib import Path
import csv
import xlrd
import io
from zipfile import ZipFile
from datetime import date, timedelta, time, datetime

import matplotlib.pyplot as plt

In [2]:
today_yyyymmdd = date.today().strftime("%Y%m%d")
print(today_yyyymmdd)

20200804


import datetime 
base = datetime.date(2020,2,1)
for i in range(0,10): 
    d=base + datetime.timedelta(i)
    d=d.strftime("%Y%m%d")
    print(f"N029kakutokuruiseki{d}.csv")

from datetime import datetime, timedelta

def date_range(start_date: datetime, end_date: datetime):
    diff = (end_date - start_date).days + 1
    return (start_date + timedelta(i) for i in range(diff))

In [3]:
#pandasの表示列数を増やす
import pandas as pd
pd.set_option('display.max_columns', 100)

In [4]:
data_dir = Path(r"C:\Users\crie072\Documents\01_データ関連\Adhoc\202007_取引価額")

#input
new_cust = "summary_credit.csv"

#out_put
result = "年間利用金額分布"

print(new_cust)
print(result)

summary_credit.csv
年間利用金額分布


column_names = ['実績月','入会申込番号','直近３ケ月買物利用金額','直近３ケ月全体利用金額','年令','性別',
                '都道府県名','顧客状況コ－ド','マイペ設定','DM希望店','カード券種名','カ－ド加入日','初回利用日',
                'あとからリボ申込有無コード','Weｂ明細有無コード','解約','入会チャネル']

新規入会客の集計

In [35]:
df = pd.read_csv(data_dir /r"input"/f"{new_cust}",encoding='shift-jis')
df = df.drop(columns={'no'})
df.head()

,kanyu_yymm,card,kzk_f,tot_credit_amt,tot_genkin_amt,tot_amt,tot_credit_trn,tot_genkin_trn,tot_trn
0,201803,ゴールド,0,2121870,0,2121870,39,0,39
1,201803,ゴールド,0,0,17820,17820,0,2,2
2,201803,ゴールド,1,184338,32647,216985,48,33,81
3,201803,ゴールド,1,168245,84965,253210,12,27,39
4,201803,ゴールド,0,878612,52056,930668,7,4,11


In [36]:
print(df.shape)
print(df.index)
print(df.columns)
print(df.dtypes)

(162415, 9)
RangeIndex(start=0, stop=162415, step=1)
Index(['kanyu_yymm', 'card', 'kzk_f', 'tot_credit_amt', 'tot_genkin_amt',
       'tot_amt', 'tot_credit_trn', 'tot_genkin_trn', 'tot_trn'],
      dtype='object')
kanyu_yymm         int64
card              object
kzk_f              int64
tot_credit_amt     int64
tot_genkin_amt     int64
tot_amt            int64
tot_credit_trn     int64
tot_genkin_trn     int64
tot_trn            int64
dtype: object


In [37]:
#2019年3月までに限定する
df =df[(df["kanyu_yymm"] <=201903)]

In [38]:
#日付データ変換
df['kanyu_yymmdd']=df['kanyu_yymm']*100+1
df['kanyu_yymmdd'] = pd.to_datetime(df['kanyu_yymmdd'], format = '%Y%m%d')
df['kanyu_year'] = df['kanyu_yymmdd'].dt.year
df['kanyu_month'] = df['kanyu_yymmdd'].dt.month
df['kanyu_yymm'] = df[['kanyu_year', 'kanyu_month']].apply(lambda x: '{}年{}月'.format(x[0], x[1]), axis=1)
df.head()

,kanyu_yymm,card,kzk_f,tot_credit_amt,tot_genkin_amt,tot_amt,tot_credit_trn,tot_genkin_trn,tot_trn,kanyu_yymmdd,kanyu_year,kanyu_month
0,2018年3月,ゴールド,0,2121870,0,2121870,39,0,39,2018-03-01,2018,3
1,2018年3月,ゴールド,0,0,17820,17820,0,2,2,2018-03-01,2018,3
2,2018年3月,ゴールド,1,184338,32647,216985,48,33,81,2018-03-01,2018,3
3,2018年3月,ゴールド,1,168245,84965,253210,12,27,39,2018-03-01,2018,3
4,2018年3月,ゴールド,0,878612,52056,930668,7,4,11,2018-03-01,2018,3


In [39]:
df['card'].unique()

array(['ゴールド', '一般', '外商'], dtype=object)

In [40]:
df['kanyu_yymm'].unique()

array(['2018年3月', '2018年4月', '2018年5月', '2018年6月', '2018年7月', '2018年8月',
       '2018年9月', '2018年10月', '2018年11月', '2018年12月', '2019年1月',
       '2019年2月', '2019年3月'], dtype=object)

In [41]:
#欠損データがないかチェック
df.isnull().sum()

kanyu_yymm        0
card              0
kzk_f             0
tot_credit_amt    0
tot_genkin_amt    0
tot_amt           0
tot_credit_trn    0
tot_genkin_trn    0
tot_trn           0
kanyu_yymmdd      0
kanyu_year        0
kanyu_month       0
dtype: int64

一般カード

In [48]:
df['tot_amt_k'] = df['tot_amt']/1000
df_ippan = df.query('card == "一般" and tot_amt_k>0')
df_ippan.shape[0]

86999

In [50]:
df_ippan['tot_amt_k'].describe()

count    86999.000000
mean       131.879939
std        297.596772
min          0.050000
25%         17.845000
50%         54.584000
75%        143.567500
max      31709.254000
Name: tot_amt_k, dtype: float64

ゴールド

In [51]:
df['tot_amt_k'] = df['tot_amt']/1000
df_gold = df.query('card == "ゴールド" and tot_amt_k>0')
df_gold.shape[0]

6296

In [52]:
df_gold['tot_amt_k'].describe()

count     6296.000000
mean       509.735040
std       1357.363526
min          0.302000
25%         65.408750
50%        205.107000
75%        560.822000
max      69584.965000
Name: tot_amt_k, dtype: float64

お得意様

In [46]:
df['tot_amt_k'] = df['tot_amt']/1000
df_otoku = df.query('card == "外商" and tot_amt_k>0')
df_otoku.shape[0]

13274

In [53]:
df_otoku['tot_amt_k'].describe()

count     13274.000000
mean        778.565872
std        2746.525070
min           0.195000
25%          49.648750
50%         224.607000
75%         762.086250
max      208910.196000
Name: tot_amt_k, dtype: float64